In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
from sklearn.metrics import recall_score,precision_score,accuracy_score,classification_report,confusion_matrix,f1_score
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.metrics import r2_score

In [150]:
df = pd.read_csv('/content/drive/MyDrive/dataset_RT_prepared.csv')

In [151]:
df['experience_years'] = df['total_experience.months'] / 12
df = df[df['experience_years'] <= df['age']]
df = df[df['salary.amount'] <= 91_000]
df = df[df['salary.amount'] >= 20_000]

In [152]:
df = df.dropna(subset=['salary.amount'])

In [153]:
columns_to_keep = ['vacancy_type',
'area_orig',
'updated_at_res',
'age',
'gender.id',
'education.level.id',
'salary.amount',
'total_experience.months',
]

In [154]:
df_clean_iqr=df

In [155]:
df_clean_iqr = df_clean_iqr[columns_to_keep]

In [156]:
df_clean_iqr['education.level.id'] = df_clean_iqr['education.level.id'].map({'secondary': 1, 'special_secondary': 2, 'bachelor': 3, 'unfinished_higher': 4,
                                                         'higher': 5, 'master' : 6, 'candidate' : 7,
                                                         'doctor' : 8})

<ipython-input-156-5249ce73c8d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_iqr['education.level.id'] = df_clean_iqr['education.level.id'].map({'secondary': 1, 'special_secondary': 2, 'bachelor': 3, 'unfinished_higher': 4,


In [157]:
numeric_cols = df_clean_iqr.select_dtypes(include=['number']).columns

df_clean_iqr[numeric_cols] = df_clean_iqr[numeric_cols].fillna(df_clean_iqr[numeric_cols].median())

<ipython-input-157-e53c75cef3b3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean_iqr[numeric_cols] = df_clean_iqr[numeric_cols].fillna(df_clean_iqr[numeric_cols].median())


In [158]:
missing_counts = df_clean_iqr.isnull().sum()
missing_counts

,0
vacancy_type,0
area_orig,4
updated_at_res,0
age,0
gender.id,0
education.level.id,0
salary.amount,0
total_experience.months,0


Тут
MSE: 249365432.4508
MAE: 12805.9754
RMSE: 15791.3088
R2 Score: 0.2245

# Adding Common Russia Macro Features (Interest Rate, Inflation)

In [159]:
df_2=df_clean_iqr

In [160]:
df_2['updated_at_res'] = pd.to_datetime(df_2['updated_at_res'])
df_2['month'] = df_2['updated_at_res'].dt.month
df_2['year'] = df_2['updated_at_res'].dt.year

<ipython-input-160-8d242cf921c5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['updated_at_res'] = pd.to_datetime(df_2['updated_at_res'])
<ipython-input-160-8d242cf921c5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['month'] = df_2['updated_at_res'].dt.month
<ipython-input-160-8d242cf921c5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [161]:
df_2=df_2.drop(columns=[ 'updated_at_res'])

In [162]:
df_3 = pd.read_csv('/content/common_russian_info_russia_inflation_monthly_year_over_year_statbureau.csv')

In [163]:
merged_df = pd.merge(df_2,
                    df_3[['year', 'month', 'inflation_rate', 'interest_rate'
                    ]],
                    on=['year', 'month'],
                    how='left')

print("Результат объединения:")
print(merged_df)

Результат объединения:
                       vacancy_type  area_orig   age gender.id  \
0                        Агент ГАП        Чита  19.0    female   
1                          Агент ТМ   Оренбург  20.0    female   
2                        Агент ГАП        Чита  32.0    female   
3                        Агент ГАП     Саратов  22.0    female   
4                        Агент ГАП     Саратов  42.0    female   
...                             ...        ...   ...       ...   
61104  Менеджер по поддержке продаж  Хабаровск  26.0      male   
61105                    Агент ГАП     Саратов  29.0      male   
61106                    Агент ГАП        Чита  29.0    female   
61107                    Агент ГАП     Иркутск  28.0      male   
61108                    Агент ГАП     Иркутск  37.0    female   

       education.level.id  salary.amount  total_experience.months  month  \
0                     2.0        40000.0                      5.0      1   
1                     2.0       

In [164]:
missing_counts = merged_df.isnull().sum()
missing_counts

,0
vacancy_type,0
area_orig,4
age,0
gender.id,0
education.level.id,0
salary.amount,0
total_experience.months,0
month,0
year,0
inflation_rate,0


Тут:
MSE: 248928115.1460
MAE: 12795.3799
RMSE: 15777.4559
R2 Score: 0.2259

# Adding Special Features of Regions of Russia

In [165]:
df_regions=pd.read_csv('/content/avg_salsry_regions_fein - с 2019 (1).csv')

In [166]:
import pandas as pd

df_regions_long = df_regions.reset_index() \
                          .melt(id_vars=['Region Name'],
                                var_name='month_year',
                                value_name='avg_salary_per_region')
df_regions_long

,Region Name,month_year,avg_salary_per_region
0,Российская Федерация,index,0.0
1,Центральный федеральный округ,index,1.0
2,Белгородская область,index,2.0
3,Брянская область,index,3.0
4,Владимирская область,index,4.0
...,...,...,...
1381,Сахалинская область,01.01.2025,151597.0
1382,Еврейская авт.область,01.01.2025,79279.0
1383,Чукотский авт.округ,01.01.2025,185029.0
1384,NaN,01.01.2025,NaN


In [167]:
df_regions['Region Name'].unique()

array(['Российская Федерация', 'Центральный федеральный округ',
       'Белгородская область', 'Брянская область', 'Владимирская область',
       'Воронежская область', 'Ивановская область', 'Калужская область',
       'Костромская область', 'Курская область', 'Липецкая область',
       'Московская область', 'Орловская область', 'Рязанская область',
       'Смоленская область', 'Тамбовская область', 'Тверская область',
       'Тульская область', 'Ярославская область', 'г.Москва ',
       'Северо-Западный федеральный округ', 'Республика Карелия',
       'Республика Коми', 'Архангельская область',
       'в том числе Ненецкий авт.округ',
       'Архангельская область без авт. округа.', 'Вологодская область',
       'Калининградская область', 'Ленинградская область',
       'Мурманская область', 'Новгородская область', 'Псковская область',
       'г.Санкт-Петербург', 'Южный федеральный округ  ',
       'Республика Адыгея', 'Республика Калмыкия', 'Республика Крым',
       'Краснодарский кр

In [168]:
df_regions_long=df_regions_long[df_regions_long['month_year']!='index']

In [169]:
df_regions_long['month_year'] = pd.to_datetime(df_regions_long['month_year'])
df_regions_long['month'] = df_regions_long['month_year'].dt.day
df_regions_long['year'] = df_regions_long['month_year'].dt.year

<ipython-input-169-1830e7693be3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regions_long['month_year'] = pd.to_datetime(df_regions_long['month_year'])
<ipython-input-169-1830e7693be3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regions_long['month'] = df_regions_long['month_year'].dt.day
<ipython-input-169-1830e7693be3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in th

In [170]:
df_reg=merged_df.dropna()

In [171]:
df_regions_oblast=pd.read_csv('/content/справочник_13.csv')

In [172]:
merged_df__reg_obl = pd.merge(df_reg,
                    df_regions_oblast[['area_orig', 'Region Name'
                    ]],
                    on=['area_orig'],
                    how='left')


In [173]:

df_avg_sal_reg = pd.merge(merged_df__reg_obl,
                    df_regions_long[[ 'month', 'year','Region Name', 'avg_salary_per_region'
                    ]],
                    on=['Region Name', 'month', 'year'],
                    how='left')


In [174]:
missing_counts = df_avg_sal_reg.isnull().sum()
missing_counts

,0
vacancy_type,0
area_orig,0
age,0
gender.id,0
education.level.id,0
salary.amount,0
total_experience.months,0
month,0
year,0
inflation_rate,0


In [175]:
df_avg_sal_reg=df_avg_sal_reg.drop_duplicates()

In [176]:
df_avg_sal_reg.shape

(59728, 13)

In [177]:
df_regions_new=pd.read_csv('/content/macro_regions_info_cursach_s.csv')

In [178]:

df_avg_sal_reg_n = pd.merge(df_avg_sal_reg,
                    df_regions_new,
                    on=['Region Name'],
                    how='left')


In [179]:
missing_counts = df_avg_sal_reg_n.isnull().sum()
missing_counts

,0
vacancy_type,0
area_orig,0
age,0
gender.id,0
education.level.id,0
salary.amount,0
total_experience.months,0
month,0
year,0
inflation_rate,0


In [180]:
df2=pd.read_csv('/content/add_new_features_cursach_33.csv')

In [181]:
df_all_features = pd.merge(df_avg_sal_reg_n,
                    df2[['Region Name',
                         'percent_children','percent_elderly','median_profit_per_person',
                         'relation_profit_prozit_minimum','natural_prirost_rozhd',
                         'mortality','median_salary','percent_with_internet','migration_prirost','number_top_school',
                         'prirost_naselenia','percent_poor','rate_of_quality_life'
                    ]],
                    on=['Region Name'],
                    how='left')

In [182]:
missing_counts = df_all_features.isnull().sum()
missing_counts

,0
vacancy_type,0
area_orig,0
age,0
gender.id,0
education.level.id,0
salary.amount,0
total_experience.months,0
month,0
year,0
inflation_rate,0


In [183]:
df_all_features=df_all_features.dropna()

In [184]:
df_all_features=df_all_features.drop_duplicates()